## Training progress

In [ ]:
import sys
import os
import pandas as pd

working_dir = '../output'
history_file = f'{working_dir}/history.csv'
if not os.path.exists(history_file):
    print(f'{history_file} not found')
    print('must train a model before running this')
    sys.exit()
    
df = pd.read_csv(history_file)

train_loss = []
val_loss = []
for _, group in df.groupby('epoch'):
    train_loss.append(group['train_loss'].mean())
    val_loss.append(group['epoch_val_loss'].mean())

loss_df = pd.DataFrame({'Training loss': train_loss, 'Validation loss': val_loss})
min_val_loss = df['epoch_val_loss'].min()
plots = loss_df.plot(xlabel='Epoch', grid=True, title=f'Minimum validation loss {min_val_loss:.4f}')

## Predictions on sample validation images

In [ ]:
import torch
from torch import nn

import numpy as np
from util import visualize
from train import Trainer
from config import Config

device = torch.device('cpu')
input_dir = '../input'
model_file = f'{working_dir}/model.pth'
if not os.path.exists(model_file):
    print(f'{model_file} not found')
    sys.exit()
checkpoint = torch.load(model_file, map_location=device)
conf = Config(checkpoint['conf'])

trainer = Trainer(conf, input_dir, device, 1, checkpoint, subset=10)
loader = trainer.val_loader
model = trainer.model

sigmoid = nn.Sigmoid()
images, labels = iter(loader).next()
model.eval()
with torch.no_grad():
    images = images.to(device)
    outputs = model(images)
    preds = sigmoid(outputs).round().cpu().numpy()

# display this many images
count = min(16, len(images))
images, labels = images.numpy(), labels.numpy()    
for idx in range(count):
    image, label, pred = images[idx], labels[idx], preds[idx]
    image = image[:3]
    visualize(image=image, label=label, prediction=pred)